In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import tensorflow as tf

Using TensorFlow backend.


In [2]:
with open('X_train_scaled_dic.pickle', 'rb') as f:
    X_train_scaled_dic = pickle.load(f)
with open('X_test_scaled_dic.pickle', 'rb') as f:
    X_test_scaled_dic = pickle.load(f)
with open('y_train_dic.pickle', 'rb') as f:
    y_train_dic= pickle.load(f)
with open('y_test_dic.pickle', 'rb') as f:
    y_test_dic= pickle.load(f)

In [3]:
with open('sector.pickle', 'rb') as f:
    sector = pickle.load(f)

# 윈도우 데이터 셋 생성

In [6]:
# 윈도우 데이터셋 생성 함수
def windowed_dataset(X, y, X_size, y_size, X_shift, y_shift, X_stride, y_stride, batch_size):
    
    # numpy array x(input)를 Dataset으로 변환
    ds_x = tf.data.Dataset.from_tensor_slices(X)
    
    #윈도우셋 구성
    ds_x = ds_x.window(size=X_size,stride=X_stride, shift=X_shift, drop_remainder=True)
    #한 차원 줄야사 반환
    ds_x = ds_x.flat_map(lambda x: x.batch(X_size))
    
    ds_y = tf.data.Dataset.from_tensor_slices(y)
    
    #같은 인덱스에 있는 애들 끼리 묶기
    ds = tf.data.Dataset.zip((ds_x, ds_y))

    
    return ds.batch(batch_size).prefetch(1)

In [7]:
## 파라미터 지정

In [8]:
#윈도우 사이즈
WINDOW_SIZE = 10
# X에 대한 윈도우 함수 설정값
X_size = WINDOW_SIZE
X_shift = 1
X_stride = 1
# y에 대한 윈도우 함수 설정값
y_size = 1
y_shift = 1
y_stride = 1

batch_size = 32

In [9]:
tf_dataset_train_dic={}
for sector_name, sector_df in X_train_scaled_dic.items():
    tf_dataset_train_dic[sector_name] = {}
    for stock_code, df in sector_df.items():
        X_train = df
        y_train = y_train_dic[sector_name][stock_code]
        tf_dataset_train_dic[sector_name][stock_code] = windowed_dataset(X_train,y_train[WINDOW_SIZE:], X_size, y_size,
                                    X_shift, y_shift, X_stride, y_stride, batch_size)

        
        
tf_dataset_test_dic={}
for sector_name, sector_df in X_test_scaled_dic.items():
    tf_dataset_test_dic[sector_name] = {}
    for stock_code, df in sector_df.items():
        X_test = df
        y_test = y_test_dic[sector_name][stock_code]
        tf_dataset_test_dic[sector_name][stock_code] = windowed_dataset(X_test,y_test[WINDOW_SIZE:], X_size, y_size,
                                    X_shift, y_shift, X_stride, y_stride, batch_size)

In [10]:
# 데이터 shape, 미리보기로 체크
for x, y in tf_dataset_train_dic['Food']['097950'].take(1):
    print('X:', x.shape)
    print(x[:3])
    print()
    print('Y:', y.shape)
    print(y[:3])
    print('-'*100)

X: (32, 10, 5)
tf.Tensor(
[[[0.08718198 0.09461967 0.08545455 0.08431373 0.08082707]
  [0.04791903 0.08163265 0.07818182 0.08627451 0.08458647]
  [0.05072111 0.08534323 0.08727273 0.08823529 0.09586466]
  [0.12175059 0.1038961  0.09636364 0.10392157 0.10338346]
  [0.09202992 0.10760668 0.10545455 0.11568627 0.11466165]
  [0.09526414 0.12244898 0.12181818 0.1254902  0.11654135]
  [0.07396998 0.11131725 0.11636364 0.11764706 0.12030075]
  [0.06126101 0.11131725 0.11272727 0.10588235 0.10526316]
  [0.07603778 0.11502783 0.10363636 0.09803922 0.10714286]
  [0.15095811 0.10760668 0.12909091 0.10588235 0.14097744]]

 [[0.04791903 0.08163265 0.07818182 0.08627451 0.08458647]
  [0.05072111 0.08534323 0.08727273 0.08823529 0.09586466]
  [0.12175059 0.1038961  0.09636364 0.10392157 0.10338346]
  [0.09202992 0.10760668 0.10545455 0.11568627 0.11466165]
  [0.09526414 0.12244898 0.12181818 0.1254902  0.11654135]
  [0.07396998 0.11131725 0.11636364 0.11764706 0.12030075]
  [0.06126101 0.11131725 0.1

In [11]:
X_train_scaled_dic['Food']['097950'].iloc[:30]

,거래량,시가,고가,저가,종가
일자,,,,,
2010-06-22,0.087182,0.094620,0.085455,0.084314,0.080827
2010-06-23,0.047919,0.081633,0.078182,0.086275,0.084586
2010-06-24,0.050721,0.085343,0.087273,0.088235,0.095865
2010-06-25,0.121751,0.103896,0.096364,0.103922,0.103383
2010-06-28,0.092030,0.107607,0.105455,0.115686,0.114662
2010-06-29,0.095264,0.122449,0.121818,0.125490,0.116541
2010-06-30,0.073970,0.111317,0.116364,0.117647,0.120301
2010-07-01,0.061261,0.111317,0.112727,0.105882,0.105263
2010-07-02,0.076038,0.115028,0.103636,0.098039,0.107143


In [13]:
y_train_dic['Food']['097950'].iloc[[10,11,12]]

,상향,하향,횡보
일자,,,
2010-07-06,0,1,0
2010-07-07,0,1,0
2010-07-08,1,0,0


In [14]:
for sector_name, sector_tf in tf_dataset_train_dic.items():
    for stock_code, tf_dataset in sector_tf.items():
        for data in tf_dataset.take(1):
            print(stock_code)
            print(f'데이터셋(X) 구성(batch_size, window_size, feature갯수): {data[0].shape}')
            print(f'데이터셋(Y) 구성(batch_size, window_size, feature갯수): {data[1].shape}')

097950
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
271560
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
000080
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
004370
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
005300
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
093050
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
020000
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
105630
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)
001070
데이터셋(X) 구

009150
데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 10, 5)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 3)


# 모델

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm


In [16]:
model_dic ={}
for sector_name, stock_list in sector.items():
    model_dic[sector_name] = {}
    for stock_code in stock_list:
        model_dic[sector_name][stock_code] = Sequential([
            # 1차원 feature map 생성
            Conv1D(filters=32, kernel_size=5,
                   padding="causal",
                   activation="relu",
                   #feature의 개수 3개
                   input_shape=[WINDOW_SIZE, 5]),
            # LSTM
            LSTM(16, activation='tanh'),
            Dense(16, activation="relu"),
            Dense(3,activation='softmax'),
        ])

In [17]:
optimizer = Adam(0.0005)
for sector_name, sector_model_dic in model_dic.items():
    for model in sector_model_dic.values():
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [18]:
# earlystopping은 10번 epoch통안 val_loss 개선이 없다면 학습을 멈춥니다.
earlystopping = EarlyStopping(monitor='val_loss', patience=10)

In [4]:
filename_dic={}
for sector_name,stock_list in sector.items():
    filename_dic[sector_name] = {}
    for stock_code in stock_list:
        filename_dic[sector_name][stock_code] = os.path.join('MC',sector_name, stock_code+'ckeckpointer.ckpt')
        
        
with open('filename_dic.pickle','wb') as f:
    pickle.dump(filename_dic,f)

In [20]:
filename_dic['Food']

{'097950': 'MC\\Food\\097950ckeckpointer.ckpt',
 '271560': 'MC\\Food\\271560ckeckpointer.ckpt',
 '000080': 'MC\\Food\\000080ckeckpointer.ckpt',
 '004370': 'MC\\Food\\004370ckeckpointer.ckpt',
 '005300': 'MC\\Food\\005300ckeckpointer.ckpt'}

In [29]:
#체크포인트를 저장    
mc_dic = {}
for sector_name, filename_sector_dic in filename_dic.items():
    mc_dic[sector_name] = {}
    for stock_code, filename in filename_sector_dic.items():
        mc_dic[sector_name][stock_code] = ModelCheckpoint(filename, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_acc',
                             verbose=1)

In [32]:
history_dic={}
# 학습+저장
for sector_name, tf_dataset_dic in tf_dataset_train_dic.items():
    history_dic[sector_name] = {}
    for stock_code, train_data in tf_dataset_dic.items():
        mc = mc_dic[sector_name][stock_code]
        test_data = tf_dataset_test_dic[sector_name][stock_code]
        history = model_dic[sector_name][stock_code].fit(train_data, 
                            validation_data=(test_data), 
                            epochs=80, 
                            callbacks=[mc, earlystopping])
        history_dic[sector_name][stock_code] = history

Epoch 1/80
75/75 [==============================] - 1s 12ms/step - loss: 0.7756 - acc: 0.5256 - val_loss: 0.7977 - val_acc: 0.5323

Epoch 00001: val_acc did not improve from 0.55102
Epoch 2/80
75/75 [==============================] - 1s 11ms/step - loss: 0.7782 - acc: 0.5067 - val_loss: 0.7918 - val_acc: 0.5408

Epoch 00002: val_acc did not improve from 0.55102
Epoch 3/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7764 - acc: 0.5256 - val_loss: 0.7906 - val_acc: 0.5425

Epoch 00003: val_acc did not improve from 0.55102
Epoch 4/80
75/75 [==============================] - 1s 11ms/step - loss: 0.7760 - acc: 0.5239 - val_loss: 0.7888 - val_acc: 0.5425

Epoch 00004: val_acc did not improve from 0.55102
Epoch 5/80
75/75 [==============================] - 1s 11ms/step - loss: 0.7763 - acc: 0.5273 - val_loss: 0.7851 - val_acc: 0.5442

Epoch 00005: val_acc did not improve from 0.55102
Epoch 6/80
75/75 [==============================] - 1s 11ms/step - loss: 0.7766 - acc: 0.5

75/75 [==============================] - 1s 11ms/step - loss: 0.8091 - acc: 0.5122 - val_loss: 0.7588 - val_acc: 0.5561

Epoch 00042: val_acc did not improve from 0.57313
Epoch 43/80
75/75 [==============================] - 1s 10ms/step - loss: 0.8090 - acc: 0.5130 - val_loss: 0.7588 - val_acc: 0.5544

Epoch 00043: val_acc did not improve from 0.57313
Epoch 44/80
75/75 [==============================] - 1s 10ms/step - loss: 0.8089 - acc: 0.5139 - val_loss: 0.7588 - val_acc: 0.5510

Epoch 00044: val_acc did not improve from 0.57313
Epoch 45/80
75/75 [==============================] - 1s 11ms/step - loss: 0.8089 - acc: 0.5147 - val_loss: 0.7588 - val_acc: 0.5527

Epoch 00045: val_acc did not improve from 0.57313
Epoch 46/80
75/75 [==============================] - 1s 10ms/step - loss: 0.8089 - acc: 0.5155 - val_loss: 0.7588 - val_acc: 0.5510

Epoch 00046: val_acc did not improve from 0.57313
Epoch 47/80
75/75 [==============================] - 1s 10ms/step - loss: 0.8088 - acc: 0.5164 - 


Epoch 00006: val_acc did not improve from 0.51701
Epoch 7/80
75/75 [==============================] - 1s 10ms/step - loss: 0.8657 - acc: 0.5151 - val_loss: 0.8984 - val_acc: 0.5119

Epoch 00007: val_acc did not improve from 0.51701
Epoch 8/80
75/75 [==============================] - 1s 9ms/step - loss: 0.8656 - acc: 0.5151 - val_loss: 0.8986 - val_acc: 0.5119

Epoch 00008: val_acc did not improve from 0.51701
Epoch 9/80
75/75 [==============================] - 1s 10ms/step - loss: 0.8656 - acc: 0.5155 - val_loss: 0.8987 - val_acc: 0.5119

Epoch 00009: val_acc did not improve from 0.51701
Epoch 10/80
75/75 [==============================] - 1s 10ms/step - loss: 0.8655 - acc: 0.5155 - val_loss: 0.8988 - val_acc: 0.5119

Epoch 00010: val_acc did not improve from 0.51701
Epoch 11/80
75/75 [==============================] - 1s 9ms/step - loss: 0.8654 - acc: 0.5155 - val_loss: 0.8989 - val_acc: 0.5136

Epoch 00011: val_acc did not improve from 0.51701
Epoch 1/80
75/75 [=====================


Epoch 00001: val_acc did not improve from 0.42177
Epoch 2/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7943 - acc: 0.6381 - val_loss: 1.1184 - val_acc: 0.4218

Epoch 00002: val_acc did not improve from 0.42177
Epoch 3/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7935 - acc: 0.6381 - val_loss: 1.1190 - val_acc: 0.4218

Epoch 00003: val_acc did not improve from 0.42177
Epoch 4/80
75/75 [==============================] - 1s 11ms/step - loss: 0.7928 - acc: 0.6381 - val_loss: 1.1195 - val_acc: 0.4218

Epoch 00004: val_acc did not improve from 0.42177
Epoch 5/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7923 - acc: 0.6381 - val_loss: 1.1213 - val_acc: 0.4218

Epoch 00005: val_acc did not improve from 0.42177
Epoch 6/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7918 - acc: 0.6381 - val_loss: 1.1223 - val_acc: 0.4218

Epoch 00006: val_acc did not improve from 0.42177
Epoch 7/80
75/75 [=====================

75/75 [==============================] - 1s 11ms/step - loss: 0.9076 - acc: 0.4656 - val_loss: 0.7567 - val_acc: 0.4796

Epoch 00001: val_acc did not improve from 0.49150
Epoch 2/80
75/75 [==============================] - 1s 10ms/step - loss: 0.9074 - acc: 0.4664 - val_loss: 0.7567 - val_acc: 0.4796

Epoch 00002: val_acc did not improve from 0.49150
Epoch 3/80
75/75 [==============================] - 1s 10ms/step - loss: 0.9072 - acc: 0.4677 - val_loss: 0.7567 - val_acc: 0.4796

Epoch 00003: val_acc did not improve from 0.49150
Epoch 4/80
75/75 [==============================] - 1s 10ms/step - loss: 0.9070 - acc: 0.4673 - val_loss: 0.7567 - val_acc: 0.4796

Epoch 00004: val_acc did not improve from 0.49150
Epoch 5/80
75/75 [==============================] - 1s 10ms/step - loss: 0.9069 - acc: 0.4673 - val_loss: 0.7567 - val_acc: 0.4796

Epoch 00005: val_acc did not improve from 0.49150
Epoch 6/80
75/75 [==============================] - 1s 10ms/step - loss: 0.9067 - acc: 0.4664 - val_l

...


Epoch 00005: val_acc did not improve from 0.48810
Epoch 6/80
75/75 [==============================] - 1s 12ms/step - loss: 0.9001 - acc: 0.4710 - val_loss: 0.7930 - val_acc: 0.4796

Epoch 00006: val_acc did not improve from 0.48810
Epoch 7/80
75/75 [==============================] - 1s 12ms/step - loss: 0.8999 - acc: 0.4723 - val_loss: 0.7930 - val_acc: 0.4711

Epoch 00007: val_acc did not improve from 0.48810
Epoch 8/80
75/75 [==============================] - 1s 10ms/step - loss: 0.8997 - acc: 0.4719 - val_loss: 0.7933 - val_acc: 0.4728

Epoch 00008: val_acc did not improve from 0.48810
Epoch 9/80
75/75 [==============================] - 1s 10ms/step - loss: 0.8996 - acc: 0.4731 - val_loss: 0.7942 - val_acc: 0.4728

Epoch 00009: val_acc did not improve from 0.48810
Epoch 10/80
75/75 [==============================] - 1s 10ms/step - loss: 0.8994 - acc: 0.4765 - val_loss: 0.7945 - val_acc: 0.4728

Epoch 00010: val_acc did not improve from 0.48810
Epoch 11/80
75/75 [===================

75/75 [==============================] - 1s 9ms/step - loss: 0.7731 - acc: 0.5281 - val_loss: 0.7365 - val_acc: 0.4762

Epoch 00010: val_acc did not improve from 0.47619
Epoch 11/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7730 - acc: 0.5281 - val_loss: 0.7366 - val_acc: 0.4762

Epoch 00011: val_acc did not improve from 0.47619
Epoch 12/80
75/75 [==============================] - 1s 9ms/step - loss: 0.7729 - acc: 0.5281 - val_loss: 0.7367 - val_acc: 0.4762

Epoch 00012: val_acc did not improve from 0.47619
Epoch 13/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7728 - acc: 0.5281 - val_loss: 0.7368 - val_acc: 0.4762

Epoch 00013: val_acc did not improve from 0.47619
Epoch 14/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7727 - acc: 0.5281 - val_loss: 0.7369 - val_acc: 0.4762

Epoch 00014: val_acc did not improve from 0.47619
Epoch 15/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7727 - acc: 0.5281 - va

75/75 [==============================] - 1s 9ms/step - loss: 0.9997 - acc: 0.4496 - val_loss: 0.9265 - val_acc: 0.4694

Epoch 00005: val_acc did not improve from 0.47619
Epoch 6/80
75/75 [==============================] - 1s 9ms/step - loss: 0.9993 - acc: 0.4563 - val_loss: 0.9248 - val_acc: 0.4677

Epoch 00006: val_acc did not improve from 0.47619
Epoch 7/80
75/75 [==============================] - 1s 9ms/step - loss: 0.9986 - acc: 0.4576 - val_loss: 0.9237 - val_acc: 0.4660

Epoch 00007: val_acc did not improve from 0.47619
Epoch 8/80
75/75 [==============================] - 1s 10ms/step - loss: 0.9980 - acc: 0.4576 - val_loss: 0.9229 - val_acc: 0.4660

Epoch 00008: val_acc did not improve from 0.47619
Epoch 9/80
75/75 [==============================] - 1s 10ms/step - loss: 0.9977 - acc: 0.4555 - val_loss: 0.9222 - val_acc: 0.4677

Epoch 00009: val_acc did not improve from 0.47619
Epoch 10/80
75/75 [==============================] - 1s 10ms/step - loss: 0.9974 - acc: 0.4563 - val_los


Epoch 00014: val_acc did not improve from 0.57313
Epoch 15/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7859 - acc: 0.5336 - val_loss: 0.7983 - val_acc: 0.5493

Epoch 00015: val_acc did not improve from 0.57313
Epoch 16/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7858 - acc: 0.5348 - val_loss: 0.7980 - val_acc: 0.5493

Epoch 00016: val_acc did not improve from 0.57313
Epoch 17/80
75/75 [==============================] - 1s 9ms/step - loss: 0.7857 - acc: 0.5332 - val_loss: 0.7974 - val_acc: 0.5578

Epoch 00017: val_acc did not improve from 0.57313
Epoch 18/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7857 - acc: 0.5336 - val_loss: 0.7971 - val_acc: 0.5561

Epoch 00018: val_acc did not improve from 0.57313
Epoch 19/80
75/75 [==============================] - 1s 10ms/step - loss: 0.7856 - acc: 0.5361 - val_loss: 0.7968 - val_acc: 0.5595

Epoch 00019: val_acc did not improve from 0.57313
Epoch 20/80
75/75 [================

75/75 [==============================] - 1s 10ms/step - loss: 0.9635 - acc: 0.4845 - val_loss: 1.0535 - val_acc: 0.4286

Epoch 00043: val_acc did not improve from 0.42857
Epoch 44/80
75/75 [==============================] - 1s 11ms/step - loss: 0.9635 - acc: 0.4836 - val_loss: 1.0536 - val_acc: 0.4286

Epoch 00044: val_acc did not improve from 0.42857
Epoch 45/80
75/75 [==============================] - 1s 11ms/step - loss: 0.9635 - acc: 0.4840 - val_loss: 1.0536 - val_acc: 0.4286

Epoch 00045: val_acc did not improve from 0.42857
Epoch 46/80
75/75 [==============================] - 1s 11ms/step - loss: 0.9634 - acc: 0.4840 - val_loss: 1.0536 - val_acc: 0.4286

Epoch 00046: val_acc did not improve from 0.42857
Epoch 47/80
75/75 [==============================] - 1s 11ms/step - loss: 0.9634 - acc: 0.4836 - val_loss: 1.0537 - val_acc: 0.4286

Epoch 00047: val_acc did not improve from 0.42857
Epoch 48/80
75/75 [==============================] - 1s 10ms/step - loss: 0.9634 - acc: 0.4845 - 

In [42]:
with open('label_ratio_dic.pickle', 'rb') as f:
    label_ratio_dic = pickle.load(f)

In [44]:
certificated_stock_dic = {}

for sector_name, sector_model in model_dic.items():
    certificated_stock_dic[sector_name] = []
    for stock_code, model in sector_model.items():
        s = model.evaluate(tf_dataset_test_dic[sector_name][stock_code])[1]
        if s>0.55:
            print(f'{stock_code}: 의 레이블 비율')
            print(np.around(label_ratio_dic[sector_name][stock_code]*100,2))
            print(f'{stock_code} : 의 정확도 : {np.round(s*100,2)}')
            certificated_stock_dic[sector_name].append(stock_code)
            

19/19 [==============================] - 0s 8ms/step - loss: 0.7777 - acc: 0.5527
097950: 의 레이블 비율
상향    49.03
하향    48.80
횡보     2.17
Name: Label, dtype: float64
097950 : 의 정확도 : 55.27
19/19 [==============================] - 0s 8ms/step - loss: 0.7905 - acc: 0.5561
090430: 의 레이블 비율
하향    48.66
상향    47.22
횡보     4.11
Name: Label, dtype: float64
090430 : 의 정확도 : 55.61
19/19 [==============================] - 0s 7ms/step - loss: 0.6392 - acc: 0.8248
003410: 의 레이블 비율
횡보    87.83
하향     6.89
상향     5.28
Name: Label, dtype: float64
003410 : 의 정확도 : 82.48
19/19 [==============================] - 0s 8ms/step - loss: 0.7874 - acc: 0.5544
010130: 의 레이블 비율
하향    50.43
상향    47.83
횡보     1.74
Name: Label, dtype: float64
010130 : 의 정확도 : 55.44
19/19 [==============================] - 0s 8ms/step - loss: 0.7820 - acc: 0.7092
047040: 의 레이블 비율
횡보    64.18
상향    18.60
하향    17.22
Name: Label, dtype: float64
047040 : 의 정확도 : 70.92
19/19 [==============================] - 0s 8ms/step - loss: 0.8748 - 

In [45]:
with open('certificated_stock_dic.pickle', 'wb') as f:
    pickle.dump(certificated_stock_dic,f)